## Portfolio Builder and Performance Back-Test

This is the main application. 

As a 'proof of concept', the application's scope is limited to analyzing the stocks in the S&P 500 in 2022. Based on a year selected by the user (from 2010 to 2021), the app gets data for the year selected from an Sqlite database 'stocks_data_process.db'. This database contains data that was previously pulled from financialmodelingprep.com using API calls. This was done in module portfolio_builder.py.

The application then does the following:

1. Use a 'formula' in module portfolio_builder_calc_ranking.py to rank the companies by Earnings Yield and Return on Tangible assets, then pick only the top 5 companies for the portfolio. portfolio_builder_calc_ranking.py is the main module to modify to change the formula in future attempts have the app pick stocks with better return on investment.

2. Calculate the performance of the portfolio of 5 stocks, assuming $2,000 were used to buy each stock, by taking the diffence in the current stock prices and previous stock price. The previous buy is assumed to be done in the month of January in the following year, since the year-end results was used to apply the formula to find the top stocks.

3. Allow the user to run a Monte Carlo simulation to get the 95% confidence level range for how the porfolio will perform in the future, again assuing the stocks by were $2,000 each.


In [19]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np; 
import matplotlib.pyplot as plt
import panel as pn
pn.extension('tabulator')
from panel.interact import interact
from bokeh.models.widgets.tables import CheckboxEditor, NumberEditor, SelectEditor, DateEditor, TimeEditor
from datetime import datetime
import hvplot.pandas

# these modules are provided with the app and must be in the same directory
from MCForecastTools import MCSimulation
import portfolio_builder_calc_ranking as cr
import FMP_Requests as fmp


## Initializations

In [20]:
# Load the environment variables from the .env file
load_dotenv()
apikey = os.getenv("FMP_API_KEY")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [21]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [22]:
# #### Global variables
year = ""
ranked_df = pd.DataFrame()
price_change_df = pd.DataFrame()
tickers = []
gain_loss = 0
gain_loss_percent = 0
stock_buy_date = "2021-01-15"
ci_lower_ten_cumulative_return = 0
ci_upper_ten_cumulative_return = 0


## Create Widgets for PyViz Panel

In [23]:
# dropdown widget for year selection
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
selected_year = pn.widgets.Select(name='select', options=years, value='2015')

In [24]:
# create Run Monte Carlo simulation button widget
mc_button = pn.widgets.Button(name='Run Monte Carlo Simulation. Please click once only. This takes about 1 minute.', button_type='primary')


In [25]:
# function to get year from user using dropdown widget
def select_year(year_selected):
    global year
    global ranked_df
    global tickers
    
    year = year_selected

    ranked_df = cr.get_ranking(year)
    
    # get top 5 companies from dataframe df
    top5_df = ranked_df.head(n=5)
    tickers = list(top5_df.index)
    
    return

# create interaction between widget and function
select_year_dropdown = interact(select_year, year_selected=selected_year)
select_year_dropdown

Column
    [0] Column
        [0] Select(name='select', options=['2010', '2011', ...], value='2015')
    [1] Row
        [0] Str(None, name='interactive03867')

In [26]:
tickers

['GILD', 'MDLZ', 'UAL', 'DOV', 'LYB']

## Function: Calculate performance and create/update widget to display performance_table in PyViz Panel

In [27]:
# function to display performance_table

@pn.depends(selected_year)
def calc_performance(year):
    
    global price_change_df
    global gain_loss
    global gain_loss_percent
    global stock_buy_date

    # get past stock prices of top 5 companies and store in stock_prices dictionary
    # assuming stocks buy is done in Jan of next year following year when ranking
    # was done
    # 
    # To take into account non-trading days, obtain stock prices over 5 days and pick the last one
    year_int = int(year)
    year_int = year_int + 1
    year_buy_str = str(year_int)
    start_date = year_buy_str + "-01-15"
    end_date = year_buy_str + "-01-20"
    past_stock_prices = {}
    date_past_stock_prices = {}
    for ticker in tickers:
        prices = fmp.get_stock_hist_daily_prices(ticker, start_date, end_date, apikey)
        price = prices['close'].iloc[-1]
        date = prices['date'].iloc[-1]
        past_stock_prices[ticker] = price
        date_past_stock_prices[ticker] = date
    
    past_stock_prices_df = pd.DataFrame.from_dict(past_stock_prices, orient="index")
    past_stock_prices_df.columns = ['past_price']
    
    date_past_stock_prices_df = pd.DataFrame.from_dict(date_past_stock_prices, orient="index")
    date_past_stock_prices_df.columns = ['date']
    stock_buy_date = date_past_stock_prices_df['date'].iloc[-1]  # set the date to buy stock

    
    # get current stock prices of top 5 companies for that date and store in stock_prices dictionary
    current_stock_prices = {}
    for ticker in tickers:
        prices = fmp.get_current_stock_price(ticker, apikey)
        price_df = pd.DataFrame(prices)
        price = price_df.iloc[0,1]
        current_stock_prices[ticker] = price
    
    current_stock_prices_df = pd.DataFrame.from_dict(current_stock_prices, orient="index")
    current_stock_prices_df.columns = ['current_price']
    
    # Calculate price changes
    price_change_df = pd.concat([current_stock_prices_df, past_stock_prices_df], axis=1)
    price_change_df['price_change'] = current_stock_prices_df['current_price'] - past_stock_prices_df['past_price']
    price_change_df['price_change_pct'] = price_change_df['price_change'] * 100 / past_stock_prices_df['past_price']
    price_change_df.index.names = ['ticker']
    
    # create/update widget to display the performanc_table in PyViz Panel
    performance_table_widget = pn.widgets.Tabulator(price_change_df, buttons={'Print': "<i class='fa fa-print'></i>"})
        
    # Calculate gain/loss with $10,000 invested equally, ie. $2000 in each stock
    amount_purchased = 2000
    gain_loss = 0
    past_price_list = price_change_df['past_price']
    current_price_list = price_change_df['current_price']
    price_change_list = price_change_df['price_change']
    
    # calculate portfolio gain_loss
    for i in range(0, len(tickers)):
        num_stocks_purchased = amount_purchased / past_price_list.iloc[i]
        gain_loss = gain_loss + (num_stocks_purchased * price_change_list.iloc[i])
    
    # store result in global variable for use by another module
    gain_loss_percent = (gain_loss / (amount_purchased * 5)) * 100

    # print (f"If $10,000 is invested equally ($2,000 ea) for each stock in January {year_buy_str}")
    # print(f"the P&L today is ${gain_loss:,.2f}, or {gain_loss_percent:.2f}%")
           
    return performance_table_widget

calc_performance(year)

Tabulator(buttons={'Print': "<i c...}, value=        current_price  ...)

## Function: Calculate portfolio performance and create/update widget to display calculated result

In [28]:
# function to display portfolio performance in PyViz Panel

@pn.depends(selected_year)
def update_portfolio_performance(year):
    next_year = str(int(year)+ 1)
    output = f"If $10,000 is invested equally ($2,000 ea) for each stock in January {next_year}, the P&L today is ${gain_loss: ,.2f}, or {gain_loss_percent: ,.2f}%"
    
    # create/update widget to display portfolio_perfomance in PyViz Panel
    portfolio_performance_widget = pn.widgets.StaticText(name='Investment Returns', value=output, width=1000)
    
    return portfolio_performance_widget

update_portfolio_performance(year)

StaticText(name='Investment Returns', value='If $10,000 is invested eq..., width=1000)

## Function: Create/update widget to display full ranking_table for selected year in PyViz Panel

In [29]:
# function to display full ranking table

@pn.depends(selected_year)
def update_ranking_table(year):
    # text_udpate_ranking_table.value = year
    
    # create/update widget to display full ranking table in PyViz Panel
    ranking_table_widget = pn.widgets.Tabulator(ranked_df, buttons={'Print': "<i class='fa fa-print'></i>"})
    
    return ranking_table_widget

update_ranking_table(year)

Tabulator(buttons={'Print': "<i c...}, pagination='local', value=          market_cap  ...)

## Function: Get historical data for stocks purchased for portfolilio, calculate cummulative returns, create plot, and create/update widget to display plot in PyViz Panel

In [30]:
@pn.depends(selected_year)
def calc_cumulative_returns(year):
    
    start_date = "2020-01-10"
    
    now = datetime.now()
    end_date = now.strftime("%Y-%m-%d")
    
    tickers.append("%5EGSPC")

    close_prices_df = pd.DataFrame()
    count = 0
    for ticker in tickers:

        # url = ("https://financialmodelingprep.com/api/v3/historical-price-full/" + ticker + "?from=" + start_date + "&to=" + end_date + "&apikey=" + apikey)
        # response = fmp.get_jsonparsed_data(url)
        # response_df = pd.DataFrame(response['historical'])
        
        response_df = fmp.get_stock_hist_daily_prices(ticker, start_date, end_date, apikey)
        response_df1 = response_df[['date','adjClose']]
        response_df1.rename(columns = {'adjClose': ticker}, inplace=True)
        response_df1.set_index('date', inplace=True)

        if count==0:
            close_prices_df = response_df1
            count = count + 1
        else:
             close_prices_df = pd.concat([close_prices_df,response_df1], axis=1)
        count = count + 1  

    tickers.pop()  # remove S&P 500 from global variable
        
    close_prices_df.rename(columns = {'%5EGSPC': "S&P500"}, inplace=True)
    
    close_prices_df.sort_index(ascending=True, inplace=True)  # sort dataframe in ascending order in index column

    # calculate daily returns
    daily_returns = close_prices_df.pct_change().dropna()
    
#     daily_returns['portfolio'] = 0
#     for i in range (0, len(tickers)):
#         daily_returns['portfolio'] += daily_returns.iloc[:,i]

#     daily_returns = daily_returns[['S&P500','portfolio']]
    
    # Calculate cumulative returns
    cumulative_returns = (1 + daily_returns).cumprod() - 1

        
    portfolio_cumulative_plot = cumulative_returns.hvplot(
        title="Cummulative Returns of Stocks and S&P 500",
        xlabel='date', 
        ylabel='price',
        figsize =(20,40)
    )
    
#    portfolio_cumulative_plot = cumulative_returns.plot(figsize=(20,10), title="Cummulative Returns of Stocks and S&P 500")
    

    
    return portfolio_cumulative_plot

calc_cumulative_returns(year)

/Users/james/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


:NdOverlay   [Variable]
   :Curve   [date]   (value)

## Run Monte Carlo Simulation for portforlio over next 10 years when user click the button the run the simulation

In [31]:
def run_mc_sim(event):
    
    global ci_lower_ten_cumulative_return
    global ci_upper_ten_cumulative_return
    
    # Set start and end dates of 3 years back from your current date
    today = datetime.today().date()
    three_years_back = today - relativedelta(years=3)
    print(today,three_years_back)

    start_date = pd.Timestamp(three_years_back, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(today, tz="America/New_York").isoformat()
    print(start_date, end_date)

    # Set timeframe to 1Day 
    timeframe = "1Day"

    portfolio_prices_df = alpaca.get_bars(
        tickers,
        timeframe,
        start = start_date,
        end = end_date
        ).df
    portfolio_prices_df   

    tickers_df=[]
    for each_symbol in tickers: 
        tickers_df.append(portfolio_prices_df[portfolio_prices_df['symbol']==each_symbol].drop('symbol', axis=1))
    portfolio_prices_df = pd.concat(tickers_df, axis=1, keys=tickers)

    # Configure the Monte Carlo simulation to forecast 10 years cumulative returns
    # The weights should be split equal weighted.
    # Run 500 samples.
    MC_ten_year = MCSimulation(
        portfolio_data = portfolio_prices_df,
        weights = [.20,.20,.20,.20,.20],
        num_simulation = 500,
        num_trading_days = 252 * 10
    )

    # Review the simulation input data
    MC_ten_year.portfolio_data.head()

    MC_ten_year.calc_cumulative_return()
    
    mc_plot = MC_ten_year.plot_simulation()

    MC_ten_year.plot_distribution()

    MC_ten_summary_statistics = MC_ten_year.summarize_cumulative_return()

    # Review the 10-year Monte Carlo summary statistics
    print(MC_ten_summary_statistics)

    # Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
    total_portfolio = 10000
    ci_lower_ten_cumulative_return = MC_ten_summary_statistics[8] * total_portfolio
    ci_upper_ten_cumulative_return = MC_ten_summary_statistics[9] * total_portfolio

  
    mc_result_text = f"There is a 95% chance that the portfolio over the next 10 years will end within in the range of $ {ci_lower_ten_cumulative_return: ,.2f}  and $ {ci_upper_ten_cumulative_return: ,.2f}."
    
    print(mc_result_text)
    
    # mc_result_widget = pn.widgets.StaticText(name='Monte Carlo Simulation Result', value=mc_result_text, width=1000)
    
    return

mc_button.on_click(run_mc_sim)
mc_button

Button(button_type='primary', name='Run Monte Carlo S...)

## Create/update widgets to display result of the Monte Carlo simulation in PyViz Panel

In [32]:
def mc_result():
    mc_result_text = f"There is a 95% chance that the portfolio over the next 10 years will end within in the range of ${ci_lower_ten_cumulative_return: ,.2f} and ${ci_upper_ten_cumulative_return: ,.2f}."
    
    print(mc_result_text)
    
    mc_result_widget = pn.widgets.StaticText(name='Result of Monte Carlo Simulation', value=mc_result_text, width=1000)
    
    return mc_result_widget

mc_result()

There is a 95% chance that the portfolio over the next 10 years will end within in the range of $ 0.00 and $ 0.00.


StaticText(name='Result of Monte C..., value='There is a 95% c..., width=1000)

## Panel Dashboard

In [33]:
#Layout using Template
template = pn.template.FastListTemplate(
    
    title='Stocks Portfolio Builder and Performance Test', 
    
    sidebar=[pn.pane.Markdown("### The aim of this application is to build a stocks portfolio, based on the year-end results of companies in a specific year, using a set of rules. This makes portfolio building more objective. The list of stocks used in this app is from the S&P 500. The rule used in this module ranks stocks by **Earnings_Yield** and **Return_on_Tangible_Assets**. The top 5 ranked stocks are used to built the portfolio."), 
             pn.pane.Markdown("## Year"),   
             selected_year],
    
    main=[pn.Row(pn.pane.Markdown("## Top 5 stocks and cumulative returns", width=1000)),
          pn.Row(pn.Column(calc_performance),
                 pn.Column(calc_cumulative_returns)),
          pn.Row(pn.pane.Markdown("## Portfolio investment return", width=1000)),
          pn.Row(pn.Column(update_portfolio_performance)),
          pn.Row(pn.pane.Markdown("## Full ranked list of stocks", width=800)),
          pn.Row(pn.Column(update_ranking_table)),
          pn.Row(pn.Column(mc_button)),
          pn.Row(pn.Column(mc_result))
          
         ],
    
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()


/Users/james/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


There is a 95% chance that the portfolio over the next 10 years will end within in the range of $ 0.00 and $ 0.00.
Launching server at http://localhost:58489
